# ENGSCI 700 Geothermal Reservoir Optimisation

This workbook is for extracting Contact well data and recreating the plots.

(Unix) launch with `cd src` >`jupyter notebook`

File structure: 
```
(root)
├── src
│    └── Python Test.ipynb
└── wairakei_data
     └── Liquid wells (version 1).xlsx
```

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
import pickle
import scipy.stats as stats
from datetime import datetime, timedelta
from plotnine import *
%matplotlib inline

base_year = '2000'    # numeric dates calculated from Jan-01

def datetime_to_numeric(my_datetime):
    # returns days since base_year-01-01.
    try:
        date_numeric = (my_datetime - datetime(int(base_year),1,1)) / timedelta(days=1)   # datetime implem
    except:
        date_numeric = (my_datetime - np.datetime64(base_year)) / np.timedelta64(1, 'D')  # numpy implem
    return date_numeric

# Check if Excel file is already in memory (loading is slow)
try:    xl
except: xl = pd.ExcelFile('../wairakei_data/Liquid wells (version 1).xlsx')
print("Sheets:", ', '.join([x for x in xl.sheet_names if set(x) & set('FtT(L') == set()]))

## Prepare data

In [ ]:
# sheets to load data from
sheets = ['wk255', 'wk256', 'w258', 'w259']
# sheets = ['wk247', 'w253', 'w254', 'w258', 'w259', 'w267', 'w268', 'wk255', 'wk256', 'w269', 'WK270', 'WK271', 'WK272']
# wells = ['wk255', 'wk256', 'wk247', 'w267', 'w268',
#          'w269', 'WK270', 'WK271', 'WK272']
        # missing: wk251, wk250, wk252, wk238, wk234, wk240, wk241, wk267, wk268

dfs = []
for sheet in sheets:
    df = xl.parse(sheet)                                       # select well data
    df['well'] = sheet                                            # label data with well name
    dfs.append(df)
df = pd.concat(dfs)

# df = df.loc[:, ~df.columns.str.contains('^Unnamed|SUMMARY|slope|intercept')]     # remove extra columns
df = df[['date', 'whp', 'mf', 'h', 'well']]                      # instead, only keep certain columns
df['well'] = df['well'].str.lower()                              # remove 'WK' inconsistencies
df['well'] = df['well'].str.replace("^[^\d]*", "wk")
df['mf'] = pd.to_numeric(df['mf'], errors='coerce')              # remove 'dummy' entries
df = df.dropna(subset=['date', 'whp', 'mf'])                     # remove NA

df['date_numeric'] = datetime_to_numeric(df['date']) #  yrs since base_year
df = df.reset_index(drop=True)
wells = df['well'].unique()
df.head()

## Set up regression data and create prediction frame for plotting

In [ ]:
date_pred = np.arange(df['date'].min(), df['date'].max(), np.timedelta64(365*2, 'D').astype(datetime))
date_numeric_pred = datetime_to_numeric(date_pred)
whp_pred = np.linspace(0, 16, 1000)
well_pred = wells
pred = pd.DataFrame(list(itertools.product(date_numeric_pred, whp_pred, well_pred)),
                    columns=['date_numeric', 'whp', 'well'])
pred.head(3)

## Perform regression and prediction

In [ ]:
from statsmodels.formula.api import ols

# Not conditioned on date
model1 = ols("mf ~ well * whp", data=df)
results1 = model1.fit()
pred['mf1'] = results1.predict(pred)

# Linear fit dependent on date
model2 = ols("mf ~ well * (whp + date_numeric)", data=df)
results2 = model2.fit()
pred['mf2'] = results2.predict(pred)

# Elliptic fit dependent on date
model3 = ols("np.power(mf,2) ~ well * (np.power(whp,2) + date_numeric)", data=df)
results3 = model3.fit()
pred['mf3^2'] = results3.predict(pred)
pred.loc[pred['mf3^2'] < 0, 'mf3^2'] = np.nan    # remove invalid results
pred['mf3'] = np.sqrt(pred['mf3^2'])

pred.head(3)

## Create plots

In [ ]:
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase

# ===============================================================
# Set up axes
# ===============================================================

# colors
cmap = plt.get_cmap('viridis')
indices = np.linspace(0, cmap.N, len(df))
my_colors = [cmap(int(i)) for i in indices]

# subplots
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=[14,4], gridspec_kw={'width_ratios': [9,9,9,1]})
ax1.get_shared_y_axes().join(ax1, ax2, ax3)
ax1.set_ylim([0, 1000])
ax1.set_xlim(0, 16)
ax1.set_ylabel('Mass flow')
ax1.set_xlabel("Well head pressure")
ax1.set_title('$mf \sim whp$')
ax2.set_title('$mf \sim whp + date$')
ax3.set_title('$mf^2 \sim whp^2 + date$')

# create date colorbar
indices = np.linspace(0, cmap.N, len(date_pred))
my_colors = [cmap(int(i)) for i in indices]
norm = Normalize(np.min(df['date']).year, np.max(df['date']).year)
cb = ColorbarBase(ax4, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label('Year')

linestyles = itertools.cycle(('-', '--', '-.', ':'))
marker = itertools.cycle(['o', ',', '+', 'x', '*', '.'])

# ===============================================================
# Plot data
# ===============================================================

# plot raw data points
for well in wells:
    mkr = next(marker)
    for ax in [ax1, ax2, ax3]:
        ax.scatter('whp', 'mf', c='date_numeric', data=df.loc[df['well']==well], marker=mkr, label=well)
    
# plot fitted curves
for well in wells:
    lty = next(linestyles)
    # model 1
    # 'data' argument filters the data to just the data from one well
    ax1.plot('whp', 'mf1', lty, data=pred[(pred['well']==well)])

    # model 2 & 3
    for i, date in enumerate(date_numeric_pred):
        # 'data' argument similar, for a specific prediction date in the loop
        ax2.plot('whp', 'mf2', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==date)], c=my_colors[i])
        ax3.plot('whp', 'mf3', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==date)], c=my_colors[i])

# show model selection criteria
for ax, result in zip([ax1, ax2, ax3], [results1, results2, results3]):
    ax.legend(['Adj $R^2$: %.2f' % result.rsquared_adj,
               'AIC: %.2f' % result.aic], 
              handlelength=0, handletextpad=0, loc=1).legendHandles[0].set_visible(False)

## Set up PyJAGS

In [ ]:
import pyjags
import warnings
from sklearn.model_selection import ParameterGrid


class MyModel(pyjags.Model):
    """
    Create my own model child class that doesn't ValueError on unused variables
    """
    def _init_compile(self, data, generate_data):
        if data is None:
            data = {}
        data = pyjags.model.dict_to_jags(data)
        unused = set(data.keys()) - set(self.variables)
        if unused:
#             warnings.warn('Unused data for variables: {}'.format(','.join(unused)))
            pass
        self.console.compile(data, self.chains, generate_data)
        

def make_prediction_data(whp_pred):
    data_df = df.copy()
    date_numeric_pred = [datetime_to_numeric(datetime.now())]
    try: iter(whp_pred)
    except TypeError: whp_pred = [whp_pred]
    else: whp_pred = list(whp_pred)
    mf_pred = [np.nan]
    well_pred = df['well'].unique()
    X = list(ParameterGrid({'date_numeric': date_numeric_pred,
                            'whp': whp_pred,
                            'mf': mf_pred,
                            'well': well_pred}))
    empty_rows = {key: [x[key] for x in X] for key in X[0].keys()}
    
    data_df = data_df.append(pd.DataFrame(empty_rows)).reset_index()
    data = data_df.to_dict('list')
    data['well_id'], unique_wells = data_df['well'].factorize()
    data['well_id'] += 1
    data['m'] = len(unique_wells)
    data['n_init'] = len(df)
    data['n'] = len(data_df)
    for key in ['index', 'h', 'date', 'well']:
        # rm unused variables
        del data[key]
    # prepare np.nans for prediction
#     data = {key: np.ma.masked_invalid(values) for key, values in data.items()}
    data['mf'] = np.ma.masked_invalid(data['mf'])
    return data

def run_model(data, n_chains=1, burn_in=100, n_samples=1000, variables=[]):
    # variables: pass in a list of variables to track (if available in data)
    model = MyModel(code, data=data, chains=n_chains, progress_bar=False)
    model.sample(burn_in)
    samples = model.sample(n_samples, vars=[x for x in variables if x in data.keys()])
    samples = {key: values.squeeze() for key, values in samples.items()}
    
    # create sample frame with info from data
    return_data = {key: [] for key in data.keys() if np.size(data[key]) == data['n']}
    for key in return_data.keys():
        return_data[key] = np.array(data[key][data['n_init']:data['n']])
    return_data['mf'] = samples['mf'][data['n_init']:data['n']]
    
    # add back metadata
    return_data['n'] = data['n'] - data['n_init']
    return_data['m'] = data['m']
    return_data['n_samples'] = n_samples
    
    return return_data

In [ ]:
from ipywidgets import interact, FloatSlider

code = '''
data {
  # find means
  mean_whp <- mean(whp)
  mean_date_numeric <- mean(date_numeric)
  
  # transform to centered covariates
  c_whp <- whp - mean_whp
  c_date_numeric <- date_numeric - mean_date_numeric
}
model {
  # fit individual regressions to wells
  for (i in 1:n) {
    #mu[i] <- Intercept[well_id[i]] + sqrt(max(mu2[i], 0)) + beta_date_numeric[well_id[i]] * c_date_numeric[i]
    #mu2[i] <- beta_whp[well_id[i]] * pow(c_whp[i], 2) 
    mu[i] <- Intercept[well_id[i]] + beta_whp[well_id[i]] * c_whp[i] + beta_date_numeric[well_id[i]] * c_date_numeric[i]
    mf[i] ~ dnorm(mu[i], tau)
  }

  I ~ dnorm(0, 1e-8)
  b_w ~ dnorm(0, 1e-8)
  b_d_n ~ dnorm(0, 1e-8)
  
  for (j in 1:m) {
    Intercept[j] ~ dnorm(0, 1e-8)
    beta_whp[j] ~ dnorm(0, 1e-8)
    beta_date_numeric[j] ~ dnorm(0, 1e-6)
  }

  tau ~ dgamma(1e-6, 1e-6)

  # make predictions
  wk255_pred <- mu[n-1]
  wk256_pred <- mu[n]
}
'''

def plot_everything(whp=None):
    fig, (ax, bar) = plt.subplots(1, 2, figsize=[12,4], gridspec_kw={'width_ratios': [19,1], 'wspace': 0.01})
    ax.set_ylim([0, 1000])
    ax.set_title('$mf^2 \sim whp^2 + date$')
    ax.set_ylabel('Mass flow')
    ax.set_xlabel("Well head pressure")
    ax.set_xlim(0, 16)

    # create date colorbar
    indices = np.linspace(0, cmap.N, len(date_pred))
    my_colors = [cmap(int(i)) for i in indices]
    norm = Normalize(np.min(df['date']).year, np.max(df['date']).year)
    cb = ColorbarBase(bar, cmap=cmap, norm=norm, orientation='vertical')
    cb.set_label('Year')
    linestyles = itertools.cycle(('-', '--', '-.', ':'))
    marker = itertools.cycle(['o', ',', '+', 'x', '*', '.'])
    
    # plot raw data points
    marker.__init__()
    for well in wells:
        mkr = next(marker)
        ax.scatter('whp', 'mf', c='date_numeric', data=df.loc[df['well']==well], marker=mkr, label=well)

    # plot linreg lines
    linestyles = itertools.cycle(('-', '--', '-.', ':'))
    for well in wells:
        lty = next(linestyles)
        for i, date in enumerate(date_numeric_pred):
            plotdata = pred[(pred['well']==well) & (pred['date_numeric']==date)]
            ax.plot('whp', 'mf3', lty, data=plotdata,
                    c=my_colors[i], alpha=0.5)

    # make and plot predictions of future measurements
    data = make_prediction_data(whp)
    samples = run_model(data, variables=['mf'])
    for i in range(samples['n']):
        ax.violinplot(samples['mf'][i], positions=[samples['whp'][i]], showmeans=True, showextrema=False)
#     fig.savefig('this.png', dpi=300, bbox_inches='tight')

# interact(plot_everything, whp=FloatSlider(min=0, max=16, value=12, continuous_update=False));

## Introduce Flash Plants

In [ ]:
# import and process data
try:    fpxl
except: fpxl = pd.ExcelFile('../wairakei_data/Data for AU.xlsx')
    
fpdf = pd.read_excel(fpxl, 'calculation', header=1, usecols="D:E, J:L, N:P")
fpdf = fpdf.rename(columns={"FP15": "well", "Unnamed: 1": "fp",
                            "hf": "hf_ip", "hg": "hg_ip", "hfg": "hfg_ip",
                            "hf.1": "hf_lp", "hg.1": "hg_lp", "hfg.1": "hfg_lp"})
fpdf = fpdf[pd.to_numeric(fpdf['hf_ip'], errors='coerce').notnull()]  # make sure it has the necessary data
for col in ['well', 'fp']:
    fpdf[col] = fpdf[col].str.lower()
fpdf[fpdf.columns] = fpdf[fpdf.columns].apply(pd.to_numeric, errors='ignore')

In [ ]:
# find details of the last known operating conditions
last_idx = df.groupby('well')['date_numeric'].idxmax()
last_measurement = df.iloc[last_idx][['well', 'date_numeric', 'whp', 'h']]

# set mapping of wells to FPs
well_fp_map = pd.DataFrame({'well': ['wk247', 'wk253', 'wk254', 'wk255', 'wk256', 'wk258', 'wk259', 'wk267', 'wk268', 'wk269', 'wk270', 'wk271', 'wk272'],
                            'fp':   ['fp15', 'fp16',  'fp16',  'fp15',  'fp15',  'fp16',  'fp16',  'fp16',  'fp16',  'fp15',  'fp15',  'fp15',  'fp15']})
# remove absent wells from map
well_fp_map = well_fp_map[well_fp_map['well'].isin(wells)]

last_flows = last_measurement.merge(well_fp_map.merge(fpdf))
well_fp_map

In [ ]:
code_fp = '''
data {
  # find means
  mean_whp <- mean(whp)
  mean_date_numeric <- mean(date_numeric)
  
  # transform to centered covariates
  c_whp <- whp - mean_whp
  c_date_numeric <- date_numeric - mean_date_numeric
  
  c_whp_pred <- whp_pred - mean_whp
  c_date_numeric_pred <- date_numeric_pred - mean_date_numeric
}
model {
  # fit individual regressions to wells
  for (i in 1:n_data) {
    mu[i] <- Intercept[well_id[i]] + beta_whp[well_id[i]] * c_whp[i] + beta_date_numeric[well_id[i]] * c_date_numeric[i]
    mf[i] ~ dnorm(mu[i], tau)
  }
  
  for (j in 1:n_wells) {
    Intercept[j] ~ dnorm(0, 1e-6)
    beta_whp[j] ~ dnorm(0, 1e-8)
    beta_date_numeric[j] ~ dnorm(0, 1e-6)
  }
  tau ~ dgamma(1e-6, 1e-6)
  
  for (i in 1:n_wells) {
    mu_pred[i] <- Intercept[i] + beta_whp[i] * c_whp_pred[i] + beta_date_numeric[i] * c_date_numeric_pred
    #mf_pred[i] ~ dnorm(mu_pred[i], tau)
    mf_pred[i] <- mu_pred[i]
  }
  
  # estimate steam at FPs using weighted sums
  for (k in 1:n_fps) {
    mf_fp[k] <- sum(mf_pred[map[k, 1:n_fp_inflows[k]]])
    h_fp[k] <- sum(mf_pred[map[k, 1:n_fp_inflows[k]]] * h[map[k, 1:n_fp_inflows[k]]]) / max(mf_fp[k], 1e-4)
    #h_fp[k] <- sum(mf_pred[map[k, 1:n_fp_inflows[k]]] * h[map[k, 1:n_fp_inflows[k]]]) / mf_fp[k]
  }
}
'''

In [ ]:
def make_data(df=None, flows=None, well_fp_map=None, date_numeric_pred=datetime_to_numeric(datetime.now())):
    # deal with well data
    data_df = df.copy()
    data_df['well_id'], unique_wells = data_df['well'].factorize()
    data_df['well_id'] += 1
    unique_wells_dict = {k: v+1 for v, k in enumerate(unique_wells)}
#     print(data_df)
    
    # set up data for flash plant
    flows_df = flows.copy().rename(columns={'mf': 'mf_fp', 'whp': 'whp_pred'}).drop('date_numeric',1)
    flows_df['fp_id'], unique_fps = flows_df['fp'].factorize()
    flows_df['fp_id'] += 1
    unique_fps_dict = {k: v+1 for v, k in enumerate(unique_fps)}
    flows_df = flows_df.drop(['well', 'fp'], 1)
#     print(flows_df)
    
    # each fp draws from which wells: {fp: [wells]}
    map_df = well_fp_map.copy()
    map_df['fp_id'] = map_df['fp'].replace(unique_fps_dict)
    map_df['well_id'] = map_df['well'].replace(unique_wells_dict)
    map_df = map_df.groupby('fp_id')['well_id'].apply(list).to_frame()
    map_dict = {}
    inflows = map_df['well_id'].tolist()
    map_dict['n_fp_inflows'] = [len(x) for x in inflows]
    map_dict['map'] = np.zeros(shape=(len(inflows), max(map_dict['n_fp_inflows'])), dtype=int)
    for i, inflows in enumerate(inflows):
        map_dict['map'][i,0:map_dict['n_fp_inflows'][i]] = inflows
#     print("Map", map_dict)
    
    # set up stochastic nodes for fps
    fp_dict = flows_df.to_dict('list')
    fp_dict['n_fps'] = len(unique_fps)
    fp_dict['mf_pred'] = np.ma.masked_invalid([np.nan]*len(unique_wells))
    fp_dict['date_numeric_pred'] = date_numeric_pred
#     print("FP", flows_df)
    
    data = data_df.to_dict('list')
    data['n_wells'] = len(unique_wells)
    data['n_data'] = len(data_df)
    data['mf_pred'] = np.ma.masked_invalid([np.nan])
    metadata = {}
    metadata['unique_wells'] = list(unique_wells)
    metadata['unique_fps'] = list(unique_fps)
    
    data.update(map_dict)
    data.update(fp_dict)
    
    for key in ['date', 'well']:
        del data[key]
    return data, metadata
    
make_data(df, last_flows, well_fp_map)[0]

In [ ]:
n_chains = 1
burn_in = 500

data_fp, metadata = make_data(df, last_flows, well_fp_map)
model_fp = MyModel(code_fp, data=data_fp, chains=n_chains, progress_bar=False)
model_fp.sample(burn_in)
samples_fp = model_fp.sample(10000, vars=['mf_pred', 'mf_fp', 'h_fp'])

outframe_fp = pd.DataFrame(samples_fp['mf_fp'].squeeze().T)
outframe_fp.columns = metadata['unique_fps']
outframe_fp = outframe_fp.melt(var_name='name')
outframe_fp['facility'] = 'Flash plants'

outframe_h_fp = pd.DataFrame(samples_fp['h_fp'].squeeze().T)
outframe_h_fp.columns = metadata['unique_fps']
outframe_h_fp = outframe_h_fp.melt(var_name='name')
outframe_h_fp['facility'] = 'FP (enthalpy)'

outframe_well = pd.DataFrame(samples_fp['mf_pred'].squeeze().T)
outframe_well.columns = metadata['unique_wells']
outframe_well = outframe_well.melt(var_name='name')
outframe_well['facility'] = 'Wells'

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[14,4])
for name in metadata['unique_wells']:
    sns.distplot(outframe_well['value'].loc[outframe_well['name']==name], ax=ax1)
ax1.set_xlabel("Mass flow")
ax1.legend(metadata['unique_wells'])
for name in metadata['unique_fps']:
    sns.distplot(outframe_fp['value'].loc[outframe_fp['name']==name], fit=stats.norm, ax=ax2)
ax2.set_xlabel("Mass flow")
ax2.legend(metadata['unique_fps'])
for name in metadata['unique_fps']:
    sns.distplot(outframe_h_fp['value'].loc[outframe_fp['name']==name], fit=stats.norm, ax=ax3)
ax3.set_xlabel("Enthalpy")
ax3.legend(metadata['unique_fps'])

outframe_all = outframe_fp.append(outframe_well).append(outframe_h_fp)
# ggplot(outframe_all, aes(x='value', fill='name')) + geom_density(alpha=0.25) + facet_wrap('~facility', scales='free')
# ggplot(outframe_fp, aes(x='mf', fill='fp')) + geom_density(alpha=0.25)
ggplot(outframe_well, aes(x='value')) + geom_density(alpha=0.25)

In [ ]:
code = '''
model {
    z <- x - 10
    a <- sum(x*x)
    for (i in 1:2) {
        b[i] <- sum(x[y[i,1:n_col[i]]])
    }
}
'''

data = {'x': [10,20,40], 'y': np.array([[1,2], [1,3]]), 'n_col': [2,2]}
model = MyModel(code, data, chains=1)
model.sample(1, vars=['a'])